In [ ]:
#Import packages
import numpy as np 
import pandas as pd
import os
import cv2
from skimage import measure
from glob import glob
import imutils
import matplotlib.pyplot as plt

from imutils import contours
from PIL import Image as Img
from IPython.display import Image

from joblib import Parallel, delayed
from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm

In [ ]:
import shutil
import random

In [ ]:
#Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ds = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/segmentation/raw/*.jpg'))
print(len(ds))

301


In [ ]:
dd = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/classification/raw/*.jpg'))
print(len(dd))

308


In [ ]:
print(dd[1])

/content/drive/MyDrive/DL-T Cell Segmentatiom/classification/raw/H2_20210401_10to1_40min_2_08.jpg


In [ ]:
def split_donors(filenames):
  """
  this function splits the patch images by donors according to their names
  """
  num = len(filenames)
  donor_name = []


        
  for i in range(num):
    curr_name = filenames[i].split('/')[-1]
    
    donors = curr_name.split('_')[:3]
  
    donor_name.append(donors)

        
  return donor_name

In [ ]:
# dn_class = split_donors(dc)
# dn_seg = split_donors(ds)
dn_dd = split_donors(dd)

In [ ]:
df_dn_dd = pd.DataFrame(dn_dd, columns = ['donor name', 'date', 'stiffness'])
df_dn_dd.value_counts()

donor name  date      stiffness
H3          20210923  1to3         13
            20210924  10to1        11
LK18        20210305  3to1         11
LK67        20210312  1to3         11
LK51        20210319  3to1         11
H3          20210923  3to1         11
                      10to1        10
LK2         20210305  1to3         10
                      10to1        10
LK18        20210305  1to3         10
                      10to1        10
H3          20210924  3to1         10
                      1to3         10
LK51        20210319  1to3         10
H3          20210917  3to1         10
LK51        20210319  10to1        10
H3          20210917  1to3         10
                      10to1        10
LK63        20210319  1to3         10
LK67        20210312  3to1         10
H2          20210401  3to1         10
                      1to3         10
                      10to1        10
LK2         20210305  3to1         10
LK75        20210722  10to1         8
                  

In [ ]:
j_patch = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/imagej_patch_for_classification/*.jpg'))
print(len(j_patch))

8699


In [ ]:
patch_status = []
patch_id = []
for i in range(len(j_patch)):
  curr_status = j_patch[i].split('/')[-1][0:4]
  patch_status.append(curr_status)
  patch_id.append(i+1)


In [ ]:
df = pd.DataFrame(list(zip(patch_status, patch_id)), columns =['disease', 'id'])

In [ ]:
df['disease'].value_counts()

H3_2    2413
H2_2    1757
LK18    1084
LK2_     896
LK67     783
LK51     777
LK63     377
H1_2     346
LK75     266
Name: disease, dtype: int64

In [ ]:
df.count()

disease    8699
id         8699
dtype: int64

Now find the filename indexes for each category respectively

In [ ]:
lk2_idx = np.array(df.loc[df['disease'] == 'LK2_', 'id'])-1
lk18_idx = np.array(df.loc[df['disease'] == 'LK18', 'id'])-1
lk51_idx = np.array(df.loc[df['disease'] == 'LK51', 'id'])-1
lk63_idx = np.array(df.loc[df['disease'] == 'LK63', 'id'])-1
lk67_idx = np.array(df.loc[df['disease'] == 'LK67', 'id'])-1
lk75_idx = np.array(df.loc[df['disease'] == 'LK75', 'id'])-1
h1_idx = np.array(df.loc[df['disease'] == 'H1_2', 'id'])-1
h2_idx = np.array(df.loc[df['disease'] == 'H2_2', 'id'])-1
h3_idx = np.array(df.loc[df['disease'] == 'H3_2', 'id'])-1

In [ ]:
def copy_files(fnames, idx_list, dst):
  """
  this function copy and paste images to the new directory according to the input image filenames
  """
  for i in range(len(idx_list)):
      old_name = fnames[idx_list[i]].split('/')[-1]
      new_dst = dst + old_name
      shutil.copyfile(fnames[idx_list[i]], new_dst)

In [ ]:
import os
os.chdir('/content/drive/MyDrive/DL-T Cell Segmentatiom/')
list_set = os.listdir()
# print((list_set))
# if list_set[9][-4:] == '.jpg':
#   print(1)

for i in list_set:
  if i[-4:] == '.jpg':
    os.remove(i)

In [ ]:
# h1_dst = '/content/drive/MyDrive/DL-T Cell Segmentatiom/H1/'
# copy_files(j_patch, h1_idx, h1_dst)

In [ ]:
H1_filenames = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/H1/*.jpg'))
print(len(H1_filenames))

346


In [ ]:
patch_YM = []
patch_idd = []
for i in range(len(H1_filenames)):
  curr_status = H1_filenames[i].split('/')[-1].split('_')[2]
  patch_YM.append(curr_status)
  patch_idd.append(i+1)


df_H1 = pd.DataFrame(list(zip(patch_YM, patch_idd)), columns =['stiffness', 'id'])
df_H1
# lk2_idx = np.array(df.loc[df_H1['disease'] == 'LK2_', 'id'])-1
h1_idd = np.array(df_H1.loc[df_H1['stiffness'] == '10to1', 'id'])-1

In [ ]:
h1_dst = '/content/drive/MyDrive/DL-T Cell Segmentatiom/H1_10to1/'
copy_files(H1_filenames, h1_idd, h1_dst)

In [ ]:
h1_10to1 = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/H1_10to1/*.jpg'))
print(len(h1_10to1))

159


In [ ]:
# h2_dst = '/content/drive/MyDrive/DL-T Cell Segmentatiom/H2/'
# copy_files(j_patch, h2_idx, h2_dst)

In [ ]:
H2_filenames = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/H2/*.jpg'))
print(len(H2_filenames))

1757


In [ ]:
patch_YM = []
patch_idd = []
for i in range(len(H2_filenames)):
  curr_status = H2_filenames[i].split('/')[-1].split('_')[2]
  patch_YM.append(curr_status)
  patch_idd.append(i+1)


df_H2 = pd.DataFrame(list(zip(patch_YM, patch_idd)), columns =['stiffness', 'id'])
df_H2
h2_idd = np.array(df_H2.loc[df_H2['stiffness'] == '10to1', 'id'])-1

h2_dst = '/content/drive/MyDrive/DL-T Cell Segmentatiom/H2_10to1/'
copy_files(H2_filenames, h2_idd, h2_dst)

In [ ]:
h2_10to1 = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/H2_10to1/*.jpg'))
print(len(h2_10to1))

494


In [ ]:
# h3_dst = '/content/drive/MyDrive/DL-T Cell Segmentatiom/H3/'
# copy_files(j_patch, h3_idx, h3_dst)

In [ ]:
H3_filenames = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/H3/*.jpg'))
print(len(H3_filenames))

2413


In [ ]:
patch_YM = []
patch_idd = []
for i in range(len(H3_filenames)):
  curr_status = H3_filenames[i].split('/')[-1].split('_')[2]
  patch_YM.append(curr_status)
  patch_idd.append(i+1)


df_H3 = pd.DataFrame(list(zip(patch_YM, patch_idd)), columns =['stiffness', 'id'])
df_H3
h3_idd = np.array(df_H3.loc[df_H3['stiffness'] == '10to1', 'id'])-1

h3_dst = '/content/drive/MyDrive/DL-T Cell Segmentatiom/H3_10to1/'
copy_files(H3_filenames, h3_idd, h3_dst)

In [ ]:
h3_10to1 = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/H3_10to1/*.jpg'))
print(len(h3_10to1))

747


In [ ]:
lk2_dst = '/content/drive/MyDrive/DL-T Cell Segmentatiom/LK2/'
copy_files(j_patch, lk2_idx, lk2_dst)

In [ ]:
LK2_filenames = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/LK2/*.jpg'))
print(len(LK2_filenames))

896


In [ ]:
patch_YM = []
patch_idd = []
for i in range(len(LK2_filenames)):
  curr_status = LK2_filenames[i].split('/')[-1].split('_')[2]
  patch_YM.append(curr_status)
  patch_idd.append(i+1)


df_LK2 = pd.DataFrame(list(zip(patch_YM, patch_idd)), columns =['stiffness', 'id'])
df_LK2
lk2_idd = np.array(df_LK2.loc[df_LK2['stiffness'] == '10to1', 'id'])-1

lk2_dst = '/content/drive/MyDrive/DL-T Cell Segmentatiom/LK2_10to1/'
copy_files(LK2_filenames, lk2_idd, lk2_dst)

In [ ]:
lk2_10to1 = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/LK2_10to1/*.jpg'))
print(len(lk2_10to1))

227


In [ ]:
lk18_dst = '/content/drive/MyDrive/DL-T Cell Segmentatiom/LK18/'
copy_files(j_patch, lk18_idx, lk18_dst)

In [ ]:
LK18_filenames = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/LK18/*.jpg'))
print(len(LK18_filenames))

1084


In [ ]:
patch_YM = []
patch_idd = []
for i in range(len(LK18_filenames)):
  curr_status = LK18_filenames[i].split('/')[-1].split('_')[2]
  patch_YM.append(curr_status)
  patch_idd.append(i+1)


df_LK18 = pd.DataFrame(list(zip(patch_YM, patch_idd)), columns =['stiffness', 'id'])
df_LK18
lk18_idd = np.array(df_LK18.loc[df_LK18['stiffness'] == '10to1', 'id'])-1

lk18_dst = '/content/drive/MyDrive/DL-T Cell Segmentatiom/LK18_10to1/'
copy_files(LK18_filenames, lk18_idd, lk18_dst)

In [ ]:
lk18_10to1 = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/LK18_10to1/*.jpg'))
print(len(lk18_10to1))

321


In [ ]:
lk51_dst = '/content/drive/MyDrive/DL-T Cell Segmentatiom/LK51/'
copy_files(j_patch, lk51_idx, lk51_dst)

In [ ]:
LK51_filenames = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/LK51/*.jpg'))
print(len(LK51_filenames))

777


In [ ]:
patch_YM = []
patch_idd = []
for i in range(len(LK51_filenames)):
  curr_status = LK51_filenames[i].split('/')[-1].split('_')[2]
  patch_YM.append(curr_status)
  patch_idd.append(i+1)


df_LK51 = pd.DataFrame(list(zip(patch_YM, patch_idd)), columns =['stiffness', 'id'])
df_LK51
lk51_idd = np.array(df_LK51.loc[df_LK51['stiffness'] == '10to1', 'id'])-1

lk51_dst = '/content/drive/MyDrive/DL-T Cell Segmentatiom/LK51_10to1/'
copy_files(LK51_filenames, lk51_idd, lk51_dst)

In [ ]:
lk51_10to1 = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/LK51_10to1/*.jpg'))
print(len(lk51_10to1))

246


In [ ]:
lk75_dst = '/content/drive/MyDrive/DL-T Cell Segmentatiom/LK75/'
copy_files(j_patch, lk75_idx, lk75_dst)

In [ ]:
LK75_filenames = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/LK75/*.jpg'))
print(len(LK75_filenames))

266


In [ ]:
patch_YM = []
patch_idd = []
for i in range(len(LK75_filenames)):
  curr_status = LK75_filenames[i].split('/')[-1].split('_')[2]
  patch_YM.append(curr_status)
  patch_idd.append(i+1)


df_LK75 = pd.DataFrame(list(zip(patch_YM, patch_idd)), columns =['stiffness', 'id'])
df_LK75
lk75_idd = np.array(df_LK75.loc[df_LK75['stiffness'] == '10to1', 'id'])-1

lk75_dst = '/content/drive/MyDrive/DL-T Cell Segmentatiom/LK75_10to1/'
copy_files(LK75_filenames, lk75_idd, lk75_dst)

In [ ]:
lk75_10to1 = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/LK75_10to1/*.jpg'))
print(len(lk75_10to1))

94


In [ ]:
new_path = '/content/drive/MyDrive/DL-T Cell Segmentatiom/tr_val_test_1donor/train/0/'
copy_files(h3_10to1, list(range(300)), new_path)

In [ ]:
train_0 = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/tr_val_test_1donor/train/0/*.jpg'))
print(len(train_0))

300


In [ ]:
new_path = '/content/drive/MyDrive/DL-T Cell Segmentatiom/tr_val_test_1donor/train/1/'
copy_files(lk18_10to1, list(range(300)), new_path)

In [ ]:
train_1 = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/tr_val_test_1donor/train/1/*.jpg'))
print(len(train_1))

300


In [ ]:
new_path = '/content/drive/MyDrive/DL-T Cell Segmentatiom/tr_val_test_1donor/val/0/'
copy_files(h2_10to1, list(range(200)), new_path)

In [ ]:
val_0 = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/tr_val_test_1donor/val/0/*.jpg'))
print(len(val_0))

200


In [ ]:
new_path = '/content/drive/MyDrive/DL-T Cell Segmentatiom/tr_val_test_1donor/val/1/'
copy_files(lk51_10to1, list(range(200)), new_path)

In [ ]:
val_1 = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/tr_val_test_1donor/val/1/*.jpg'))
print(len(val_1))

200


In [ ]:
new_path = '/content/drive/MyDrive/DL-T Cell Segmentatiom/tr_val_test_2donor/test/0/'
copy_files(h1_10to1, list(range(100)), new_path)

In [ ]:
test_0 = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/tr_val_test_2donor/test/0/*.jpg'))
print(len(test_0))

100


In [ ]:
new_path = '/content/drive/MyDrive/DL-T Cell Segmentatiom/tr_val_test_2donor/test/1/'
copy_files(lk75_10to1, list(range(100)), new_path)

IndexError: ignored

In [ ]:
test_1 = glob(os.path.join('/content/drive/MyDrive/DL-T Cell Segmentatiom/tr_val_test_2donor/test/1/*.jpg'))
print(len(test_1))

94
